In [1]:
import logging
logging.basicConfig(level=logging.INFO)

import numpy

import cicada
from cicada.active import ActiveProtocolSuite
from cicada.communicator import SocketCommunicator

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = ActiveProtocolSuite(communicator, threshold=2)

    secret = numpy.array(42) if communicator.rank == 0 else None
    secret_share = protocol.share(src=0, secret=secret, shape=())

    result = protocol.reveal(secret_share)
    log.info(f"Player {communicator.rank} revealed: {result}")

SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 revealed: 42.0
INFO:root:Player 1 revealed: 42.0
INFO:root:Player 2 revealed: 42.0


In [2]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = ActiveProtocolSuite(communicator, threshold=2)

    secret = numpy.array(42) if communicator.rank == 0 else None
    secret_share = protocol.share(src=0, secret=secret, shape=())

    # Evil player behavior here!
    if communicator.rank == 1:
        additive_share = secret_share.storage[0]
        additive_share.storage += 1
    
    result = protocol.reveal(secret_share)
    log.info(f"Player {communicator.rank} revealed: {result}")

SocketCommunicator.run(world_size=3, fn=main);

ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 0 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/Users/tshead/src/cicada-mpc/cicada/communicator/socket/__init__.py", line 784, in launch
    result = fn(communicator, *args, **kwargs)
  File "/var/folders/tl/h2xygzzn1154jzjn_n01x860001l4n/T/ipykernel_29378/2708435132.py", line 13, in main
    result = protocol.reveal(secret_share)
  File "/Users/tshead/src/cicada-mpc/cicada/active.py", line 1116, in reveal
    return self._encoder.decode(self._reveal(share, dst=dst))
  File "/Users/tshead/src/cicada-mpc/cicada/active.py", line 1033, in _reveal
    raise ConsistencyError("Secret Shares are inconsistent in the first stage")
cicada.active.ConsistencyError: Secret Shares are inconsistent in the first stage

ERROR:cicada.communicator.socket:***********************************************

In [3]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = ActiveProtocolSuite(communicator, threshold=2)

    secret = numpy.array(42) if communicator.rank == 0 else None
    secret_share = protocol.share(src=0, secret=secret, shape=())

    # Evil player behavior here!
    if communicator.rank == 1:
        additive_share = secret_share.storage[0]
        additive_share.storage += 1

    if not protocol.verify(secret_share):
        log.error(f"Player {communicator.rank} detected tampering!")
        return
        
    result = protocol.reveal(secret_share)
    log.info(f"Player {communicator.rank} revealed: {result}")

SocketCommunicator.run(world_size=3, fn=main);

ERROR:root:Player 0 detected tampering!
ERROR:root:Player 1 detected tampering!
ERROR:root:Player 2 detected tampering!


Let's attempt a more subtle tamper:

In [4]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = ActiveProtocolSuite(communicator, threshold=2)

    secret = numpy.array(42) if communicator.rank == 0 else None
    secret_share = protocol.share(src=0, secret=secret, shape=())

    # Evil player behavior here!
    if communicator.rank == 0:
        additive_share = secret_share.storage[0]
        shamir_share = secret_share.storage[1]
        
        modulus = 2**64-59
        additive_share.storage += 1
        shamir_share.storage = numpy.array((shamir_share.storage + pow(5, modulus-2, modulus)) % modulus, dtype=object)

    if not protocol.verify(secret_share):
        log.error(f"Player {communicator.rank} detected tampering!")
        return
        
    result = protocol.reveal(secret_share)
    log.info(f"Player {communicator.rank} revealed: {result}")

SocketCommunicator.run(world_size=3, fn=main);

ERROR:root:Player 0 detected tampering!
ERROR:root:Player 1 detected tampering!
ERROR:root:Player 2 detected tampering!
